In [4]:
import pandas as pd
import numpy as np
from glob import glob

In [41]:
target_columns = [
    "Negative for Pneumonia", "Typical Appearance", "Indeterminate Appearance", "Atypical Appearance"
]

In [2]:
df = pd.read_csv('/workspace/data/train_study_level.csv')

In [3]:
df.head()

,id,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,00086460a852_study,0,1,0,0
1,000c9c05fd14_study,0,0,0,1
2,00292f8c37bd_study,1,0,0,0
3,005057b3f880_study,1,0,0,0
4,0051d9b12e72_study,0,0,0,1


In [29]:
for i in image_path_list:
    if "00086460a852" in i:
        print(i)

/workspace/data/train/00086460a852/9e8302230c91/65761e66de9f.dcm


In [13]:
df.columns

Index(['id', 'Negative for Pneumonia', 'Typical Appearance',
       'Indeterminate Appearance', 'Atypical Appearance'],
      dtype='object')

In [8]:
image_path_list = glob('/workspace/data/train/*/*/*.dcm')

In [17]:
image_path_list[0]

'/workspace/data/train/eeecfd50b220/0fd96597c559/13131b0c3db4.dcm'

In [18]:
image_path_list[1000].split("/")[4]+"_study"

'386aa41785fa_study'

In [34]:
id_list = []
neg = []
typ = []
indete = []
atyp = []
dicom_path_list = []

error = []

for i in range(len(image_path_list)):
    image_path = image_path_list[i]
    study = image_path.split("/")[4]
    try:
        row = df[df["id"] == f"{study}_study"]

        id_list += [f"{study}_study"]
        neg += [row['Negative for Pneumonia']]
        typ += [row['Typical Appearance']]
        indete += [row['Indeterminate Appearance']]
        atyp += [row['Atypical Appearance']]
        dicom_path_list += [image_path]
    except:
        error.append(image_path)

In [39]:
df_train = pd.DataFrame({
    'id': id_list,
    'dicom_path': dicom_path_list
}).merge(df, on='id', how='left')

In [40]:
df_train

,id,dicom_path,Negative for Pneumonia,Typical Appearance,Indeterminate Appearance,Atypical Appearance
0,eeecfd50b220_study,/workspace/data/train/eeecfd50b220/0fd96597c55...,0,1,0,0
1,a89c1f3470e1_study,/workspace/data/train/a89c1f3470e1/c23cc53bdbf...,0,0,1,0
2,9cc2f64f0c58_study,/workspace/data/train/9cc2f64f0c58/2eebbe88278...,0,1,0,0
3,7b6c49da06db_study,/workspace/data/train/7b6c49da06db/c1fd5829f05...,0,1,0,0
4,d14080fd6f2a_study,/workspace/data/train/d14080fd6f2a/9d6098d515e...,0,1,0,0
...,...,...,...,...,...,...
6329,6bb38a2b98f0_study,/workspace/data/train/6bb38a2b98f0/70960f315ac...,0,1,0,0
6330,8501ffeadc53_study,/workspace/data/train/8501ffeadc53/0c640403081...,0,1,0,0
6331,c1ba4d912111_study,/workspace/data/train/c1ba4d912111/5d1e7b4f209...,0,1,0,0
6332,7e4059d6a0f9_study,/workspace/data/train/7e4059d6a0f9/ac80ee2496b...,1,0,0,0


In [42]:
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

mskf = MultilabelStratifiedKFold(n_splits=5, shuffle=True, random_state=516)

df_train["cv"] = -1

for cv, (train_index, test_index) in enumerate(mskf.split(df_train, df_train.loc[:, target_columns])):
    print("TRAIN:", train_index, "TEST:", test_index)
    df_train.loc[test_index, "cv"] = cv

TRAIN: [   0    1    2 ... 6330 6332 6333] TEST: [  18   21   24 ... 6324 6327 6331]
TRAIN: [   0    1    2 ... 6329 6331 6333] TEST: [  13   16   20 ... 6328 6330 6332]
TRAIN: [   1    4    5 ... 6331 6332 6333] TEST: [   0    2    3 ... 6307 6313 6319]
TRAIN: [   0    1    2 ... 6331 6332 6333] TEST: [   4    6    8 ... 6321 6325 6329]
TRAIN: [   0    2    3 ... 6330 6331 6332] TEST: [   1    5    7 ... 6316 6322 6333]


/opt/conda/lib/python3.8/site-packages/sklearn/utils/validation.py:67: FutureWarning: Pass shuffle=True, random_state=516 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  warnings.warn("Pass {} as keyword args. From version 0.25 "


In [44]:
df_train.cv.value_counts()

0    1267
4    1267
1    1267
2    1267
3    1266
Name: cv, dtype: int64

In [45]:
df_train.to_csv('/workspace/data/df_train_study_level.csv', index=False)